## Fuzzy C-Means Clustering

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("country_wise_latest.csv")

In [3]:
df.head()

,Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,Deaths / 100 Cases,Recovered / 100 Cases,Deaths / 100 Recovered,Confirmed last week,1 week change,1 week % increase,WHO Region
0,Afghanistan,36263,1269,25198,9796,106,10,18,3.50,69.49,5.04,35526,737,2.07,Eastern Mediterranean
1,Albania,4880,144,2745,1991,117,6,63,2.95,56.25,5.25,4171,709,17.00,Europe
2,Algeria,27973,1163,18837,7973,616,8,749,4.16,67.34,6.17,23691,4282,18.07,Africa
3,Andorra,907,52,803,52,10,0,0,5.73,88.53,6.48,884,23,2.60,Europe
4,Angola,950,41,242,667,18,1,0,4.32,25.47,16.94,749,201,26.84,Africa


#### Initial Variable

In [4]:
MAX_CLUSTER = 5
MAX_ITERATION = 100
MIN_ERROR = 0.00001
FIRST_ITERATION = 1

In [5]:
def train_data(cols: list):
    return df[cols]

In [6]:
def create_random(rows, max_cluster):
    np.random.seed(0)
    return np.random.dirichlet(np.ones(max_cluster),size=rows)

### Select Data for Clustering

In [7]:
label = "Country/Region"
columns = ["New cases", "New deaths", "New recovered"]
train_data = df[columns]

In [8]:
c_df = pd.DataFrame(create_random(len(train_data), MAX_CLUSTER))

In [9]:
c_colname = ["C" + str(i + 1) for i in range(MAX_CLUSTER)]
x_colname = ["X" + str(i + 1) for i in range(len(columns))]
v_colname = ["V" + str(i + 1) for i in range(MAX_CLUSTER)]
l_colname = ["L" + str(i + 1) for i in range(MAX_CLUSTER)]
cluster_name = ["CLUSTER_" + str(i + 1) for i in range(MAX_CLUSTER)]

#### Random Number

In [10]:
c_df.columns = c_colname
train_data.columns = x_colname
c_df.head()

,C1,C2,C3,C4,C5
0,0.184517,0.291178,0.214042,0.182506,0.127756
1,0.135961,0.075372,0.291200,0.434132,0.063334
2,0.268972,0.129038,0.143912,0.445445,0.012633
3,0.016742,0.003751,0.328270,0.276536,0.374701
4,0.498615,0.208154,0.080257,0.196651,0.016322


In [11]:
result = pd.DataFrame()
result["label"] = df[label]
for i, c in enumerate(c_colname):
    result[cluster_name[i]] = np.where(c_df[c] == c_df.max(axis=1), "1", "")

In [12]:
pd.set_option('max_rows', None)
result.head(187)

,label,CLUSTER_1,CLUSTER_2,CLUSTER_3,CLUSTER_4,CLUSTER_5
0,Afghanistan,,1,,,
1,Albania,,,,1,
2,Algeria,,,,1,
3,Andorra,,,,,1
4,Angola,1,,,,
5,Antigua and Barbuda,,,1,,
6,Argentina,,1,,,
7,Armenia,,,,1,
8,Australia,,,1,,
9,Austria,1,,,,


In [13]:
c_df.head()

,C1,C2,C3,C4,C5
0,0.184517,0.291178,0.214042,0.182506,0.127756
1,0.135961,0.075372,0.291200,0.434132,0.063334
2,0.268972,0.129038,0.143912,0.445445,0.012633
3,0.016742,0.003751,0.328270,0.276536,0.374701
4,0.498615,0.208154,0.080257,0.196651,0.016322


#### Cluster

In [14]:

all_cluster = {}

div = []

for i, cluster in enumerate(cluster_name):
    all_cluster[cluster] = {}
    all_cluster[cluster]["DATA"] = pd.DataFrame()
    
    all_cluster[cluster]["DATA"][c_colname[i] + "^2"] = np.power(c_df[c_colname[i]], 2)
    all_cluster[cluster]["SUM_" + c_colname[i] + "^2"] = np.sum(all_cluster[cluster]["DATA"][c_colname[i] + "^2"])
    div.append([])
    for j, x in enumerate(x_colname):
        all_cluster[cluster]["DATA"][c_colname[i] + "*" + x] = all_cluster[cluster]["DATA"][c_colname[i] + "^2"] * train_data[x]
        
        all_cluster[cluster]["SUM_" + c_colname[i] + "*" + x] = np.sum(all_cluster[cluster]["DATA"][c_colname[i] + "*" + x])
        
        all_cluster[cluster]["SUM_" + c_colname[i] + "*" + x + "_DIV_" + "SUM_" + c_colname[i] + "^2"] = all_cluster[cluster]["SUM_" + c_colname[i] + "*" + x] / all_cluster[cluster]["SUM_" + c_colname[i] + "^2"]
        
        div[i].append(all_cluster[cluster]["SUM_" + c_colname[i] + "*" + x + "_DIV_" + "SUM_" + c_colname[i] + "^2"])

In [15]:
all_cluster["CLUSTER_1"]["DATA"].head()

,C1^2,C1*X1,C1*X2,C1*X3
0,0.034047,3.608944,0.340466,0.612840
1,0.018485,2.162789,0.110912,1.164579
2,0.072346,44.564978,0.578766,54.186962
3,0.000280,0.002803,0.000000,0.000000
4,0.248617,4.475109,0.248617,0.000000


In [16]:
all_cluster["CLUSTER_2"]["DATA"].head()

,C2^2,C2*X1,C2*X2,C2*X3
0,0.084785,8.987157,0.847845,1.526121
1,0.005681,0.664670,0.034086,0.357899
2,0.016651,10.256974,0.133207,12.471548
3,0.000014,0.000141,0.000000,0.000000
4,0.043328,0.779903,0.043328,0.000000


In [17]:
all_cluster["CLUSTER_3"]["DATA"].head()

,C3^2,C3*X1,C3*X2,C3*X3
0,0.045814,4.856287,0.458140,0.824652
1,0.084798,9.921332,0.508786,5.342256
2,0.020711,12.757842,0.165686,15.512376
3,0.107761,1.077613,0.000000,0.000000
4,0.006441,0.115943,0.006441,0.000000


In [18]:
all_cluster["CLUSTER_4"]["DATA"].head()

,C4^2,C4*X1,C4*X2,C4*X3
0,0.033309,3.530713,0.333086,0.599555
1,0.188471,22.051099,1.130826,11.873669
2,0.198421,122.227398,1.587369,148.617404
3,0.076472,0.764719,0.000000,0.000000
4,0.038672,0.696091,0.038672,0.000000


In [19]:
all_cluster["CLUSTER_5"]["DATA"].head()

,C5^2,C5*X1,C5*X2,C5*X3
0,0.016322,1.730098,0.163217,0.293790
1,0.004011,0.469313,0.024067,0.252707
2,0.000160,0.098303,0.001277,0.119528
3,0.140401,1.404012,0.000000,0.000000
4,0.000266,0.004796,0.000266,0.000000


In [20]:
# print(all_cluster["CLUSTER_2"]["SUM_C2*X2_DIV_SUM_C2^2"])

#### Pusat Cluster

In [21]:
pusat_cluster = pd.DataFrame(np.array(div))
pusat_cluster.columns = x_colname
pusat_cluster.index = cluster_name
pusat_cluster.head()

,X1,X2,X3
CLUSTER_1,3419.390481,58.529735,2317.235424
CLUSTER_2,1154.220716,34.099551,794.405869
CLUSTER_3,524.323202,12.084287,484.056412
CLUSTER_4,1206.833865,30.145610,860.561005
CLUSTER_5,543.697542,19.099963,597.000924


In [22]:
cluster = {}
for i, c in enumerate(cluster_name):
    cluster[c] = pd.DataFrame()
    for x in x_colname:
        cluster[c][ "(" + x + "-" + v_colname[i] + ")^2"] = np.power(train_data[x] - pusat_cluster["X1"][i], 2)
    cluster[c]["SUM"] = cluster[c].sum(axis=1)

In [23]:
cluster["CLUSTER_2"].head()

,(X1-V2)^2,(X2-V2)^2,(X3-V2)^2,SUM
0,1.098767e+06,1.309241e+06,1.290998e+06,3.699005e+06
1,1.075827e+06,1.318411e+06,1.190763e+06,3.585000e+06
2,2.896815e+05,1.313822e+06,1.642038e+05,1.767707e+06
3,1.309241e+06,1.332225e+06,1.332225e+06,3.973692e+06
4,1.290998e+06,1.329918e+06,1.332225e+06,3.953141e+06


#### Objective Function

In [24]:
obj_function = np.power(c_df, 2)
obj_function.columns = [c + "^2" for c in c_colname]

In [25]:
for i, l in enumerate(l_colname):
    obj_function[l] = np.abs(cluster[cluster_name[i]]["SUM"]) * obj_function[obj_function.columns[i]]
obj_function["TOTAL"] = obj_function[l_colname].sum(axis=1)

In [26]:
obj_function.head()

,C1^2,C2^2,C3^2,C4^2,C5^2,L1,L2,L3,L4,L5,TOTAL
0,0.034047,0.084785,0.045814,0.033309,0.016322,1.163440e+06,313618.323622,31881.331834,135152.104781,12286.469003,1.656379e+06
1,0.018485,0.005681,0.084798,0.188471,0.004011,6.252194e+05,20366.160225,54897.210193,742216.586709,2816.918957,1.445516e+06
2,0.072346,0.016651,0.020711,0.198421,0.000160,1.926391e+06,29433.974079,6740.820201,396028.504158,53.356393,2.358648e+06
3,0.000280,0.000014,0.107761,0.076472,0.140401,9.812224e+03,55.913920,87756.267760,332294.112480,122998.060016,5.529166e+05
4,0.248617,0.043328,0.006441,0.038672,0.000266,8.688445e+06,171281.459843,5186.147303,167209.252362,230.847181,9.032353e+06


In [27]:
TOTAL = np.sum(obj_function["TOTAL"])
LAST_ERROR = np.abs(TOTAL-0)

print(TOTAL)
print(LAST_ERROR)

3924558652.92076
3924558652.92076


#### Matrix Partition U

In [28]:
# matrix_partition = np.power(cluster["CLUSTER_1"][cluster["CLUSTER_1"].columns[0:len(x_colname)]].sum(axis=1), -1)

In [29]:
matrix_partition = pd.DataFrame()
for i, c in enumerate(cluster_name):
    matrix_partition[l_colname[i]] = np.power(cluster[c][cluster[c].columns[0:len(x_colname)]].sum(axis=1), -1)
matrix_partition["LT"] = matrix_partition.sum(axis=1)

In [30]:
matrix_partition.head()

,L1,L2,L3,L4,L5,LT
0,2.926376e-08,2.703430e-07,0.000001,2.464528e-07,0.000001,0.000003
1,2.956623e-08,2.789400e-07,0.000002,2.539298e-07,0.000001,0.000004
2,3.755506e-08,5.657045e-07,0.000003,5.010273e-07,0.000003,0.000007
3,2.856457e-08,2.516551e-07,0.000001,2.301333e-07,0.000001,0.000003
4,2.861469e-08,2.529634e-07,0.000001,2.312775e-07,0.000001,0.000003


#### Update C

In [31]:
for i, c in enumerate(c_colname):
    c_df[c] = matrix_partition[l_colname[i]]/matrix_partition["LT"]

In [32]:
c_df.head()

,C1,C2,C3,C4,C5
0,0.008837,0.081638,0.433947,0.074423,0.401155
1,0.008373,0.078996,0.437449,0.071913,0.403270
2,0.005240,0.078925,0.428656,0.069901,0.417278
3,0.009919,0.087386,0.426404,0.079913,0.396378
4,0.009837,0.086960,0.426962,0.079505,0.396736


#### Cluster Result

In [33]:
result = pd.DataFrame()
result["label"] = df[label]
for i, c in enumerate(c_colname):
    result[cluster_name[i]] = np.where(c_df[c] == c_df.max(axis=1), "1", "")

In [34]:
pd.set_option('max_rows', None)
result.head(187)

,label,CLUSTER_1,CLUSTER_2,CLUSTER_3,CLUSTER_4,CLUSTER_5
0,Afghanistan,,,1,,
1,Albania,,,1,,
2,Algeria,,,1,,
3,Andorra,,,1,,
4,Angola,,,1,,
5,Antigua and Barbuda,,,1,,
6,Argentina,1,,,,
7,Armenia,,,1,,
8,Australia,,,1,,
9,Austria,,,1,,
